EMAIL SPAM DETECTION

In [ ]:
# Needed Libraries 

import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from skleran.feature_extraction.text import TfidVectorizer  # 
